In [1]:
import torch
from torch.nn import functional as F
from config import *
from encoder import Encoder
from modules_rwkv import LLM

In [2]:
def get_loss_lr(log_file):
    with open(log_file) as f:
        lines = f.readlines()

    losses = []
    lrs = []
    for line in lines:
        step = int(line.split()[0][4:-1])
        loss = float(line.split()[1][5:])
        lr = float(line.split()[3][3:])
        if len(losses) > step - 1:
            losses[step - 1] = loss
            lrs[step - 1] = lr
        else:
            losses.append(loss)
            lrs.append(lr)
    return losses, lrs

In [3]:
import matplotlib.pyplot as plt

In [4]:
from dataloader import BinaryDataset, collate_fn
from torch.utils.data import DataLoader

loader = DataLoader(
    BinaryDataset("part-2021278643.json.bin", MAX_LENGTH),
    batch_size=1,
    shuffle=False,
    num_workers=1,
    collate_fn=collate_fn
)
encoder = Encoder.from_path("encoder.json")

In [5]:
llm = LLM(encoder.vocab_size, MODEL_DIM, LORA_DIM, N_BLOCKS, N_HEADS).to("cuda")

In [6]:
length = MAX_LENGTH
for x, y, n_tokens in loader:
    x = x.to("cuda")[:, :length, ...]
    y = y.to("cuda")[:, :length, ...]
    break

In [7]:
print(f"{sum(para.numel() for para in llm.parameters())} parameters.")
torch.set_float32_matmul_precision('high')
logits, state = llm(x)
loss = F.cross_entropy(logits.contiguous().view(-1, logits.size(-1)), y.contiguous().view(-1), reduction='mean', ignore_index=0)

78255616 parameters.
torch.Size([1, 1024, 768])
torch.Size([1, 1024, 768])
torch.Size([1, 1024, 768])
torch.Size([1, 1024, 768])
torch.Size([1, 1024, 768])


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 5.70 GiB of which 2.38 MiB is free. Process 11344 has 57.60 MiB memory in use. Including non-PyTorch memory, this process has 5.57 GiB memory in use. Of the allocated memory 5.23 GiB is allocated by PyTorch, and 216.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
out = logits[0].argmax(dim=-1).tolist()[MAX_LENGTH - length:]
print(encoder.decode(x[0].tolist()))
print(encoder.decode(y[0].tolist()))
print(encoder.decode(out))

重庆通报3起违反中央八项规定精神问题：央广网重庆1月31日消息(记者吴新伟)重庆市纪委监察委官微"风正巴渝"今披露,该市近日通报3起违反中央八项规定精神问题。据通报,重庆长寿化工有限责任公司党委委员、董事、副总经理李文成违规发放津补贴。2013年4月至2017年11月,李文成违规以节日津补贴等名义向公司办公室职工发放11万余元,其中本人违规领取3.2万余元。2018年7月,李文成受到党内严重警告处分、扣减2017年全部绩效年薪处理；违纪款项已清退。垫江县原卫生和计划生育委员会党委书记、主任刘卫东违规收受礼金问题。2013年1月至2017年1月,刘卫东在任垫江县原卫生局党委书记、局长,以及垫江县原卫生和计划生育委员会党委书记、主任期间,先后收受管理服务对象礼金共计4万元；同时,刘卫东还存在其他严重违纪违法问题。2018年9月,刘卫东受到开除党籍、开除公职处分；其涉嫌犯罪问题已移送司法机关依法处理；违纪款项已收缴。时任梁平县市政园林管理局党委委员、副局长雷仁友等人公款旅游问题。2015年10月,雷仁友与时任县市政设施管理所所长王平波及该所规划科科长王斌,借参加培训之机,前往杭州、苏州等地景区旅游,并公款报销有关费用共计1万余元。2018年8月、9月,雷仁友受到党内严重警告处分,王平波、王斌均受到党内警告处分；因履行主体责任和监督责任不力,时任县市政园林管理局党委书记、局长李剑勤和时任县市政园林管理局纪委书记刘安俊被责令作出书面检查；违纪款项已收缴。<eos>真的勇!罗志祥生日全网只有他一个人送祝福,真是艺高人胆大：娱乐圈是一个很小的圈子,明星间总会有交集。罗志祥的"多人运动"把自己的名声以及事业彻底搞垮,还和周扬青分手,被粉丝和网友抵制,不得不说需要给罗志祥颁一个年度"最惨"艺人的奖。但是,这一切的事情的起因,和罗志祥的生活作风脱不了干系。这不,就算是知道罗志祥的遭到封杀,还是给罗志祥送上生日祝福的一个知名节目导演严敏,这个严敏是前四季《极限挑战》的总导演,看来严敏也是一个性情中人,就算知道罗志祥现在的状况,还是一如既往的选择支持罗志祥,不得不说,罗志祥还是人缘不错的,尤其是在事业上。其实,罗志祥要是没有伤害到周扬青的话,周扬青是不会主动曝出这些事的,对于周扬青而言,感情上的背叛是零容忍的,关键是罗志祥一直都在骗她,是个人都会忍不了。周扬青选择借助社交平台将这件事说出来

In [9]:
print(state[0].shape)

torch.Size([1, 64, 384])


In [10]:
loss.backward()